In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import tensorflow as tf
# import dask.dataframe as dd
# import pandas as pd

# test = dd.read_csv('./Data/Filtered/data-*.csv', dtype={'engine_displacement': 'object',
#        'horsepower': 'object', 'dealer_zip': 'object'})

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("./Data/used_cars_data.csv")
df.head()

C:\Users\15128\AppData\Local\Temp\ipykernel_11124\249344349.py:9: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Data/used_cars_data.csv")


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [3]:
# List all the columns in the DataFrame
columns_list = df.columns.tolist()

# Print the list of columns
print(columns_list)

['vin', 'back_legroom', 'bed', 'bed_height', 'bed_length', 'body_type', 'cabin', 'city', 'city_fuel_economy', 'combine_fuel_economy', 'daysonmarket', 'dealer_zip', 'description', 'engine_cylinders', 'engine_displacement', 'engine_type', 'exterior_color', 'fleet', 'frame_damaged', 'franchise_dealer', 'franchise_make', 'front_legroom', 'fuel_tank_volume', 'fuel_type', 'has_accidents', 'height', 'highway_fuel_economy', 'horsepower', 'interior_color', 'isCab', 'is_certified', 'is_cpo', 'is_new', 'is_oemcpo', 'latitude', 'length', 'listed_date', 'listing_color', 'listing_id', 'longitude', 'main_picture_url', 'major_options', 'make_name', 'maximum_seating', 'mileage', 'model_name', 'owner_count', 'power', 'price', 'salvage', 'savings_amount', 'seller_rating', 'sp_id', 'sp_name', 'theft_title', 'torque', 'transmission', 'transmission_display', 'trimId', 'trim_name', 'vehicle_damage_category', 'wheel_system', 'wheel_system_display', 'wheelbase', 'width', 'year']


In [7]:
df['dealer_zip'].value_counts()

77477    7318
33619    5617
77034    5255
77074    5216
91401    5113
         ... 
34208       1
19149       1
47060       1
39211       1
73013       1
Name: dealer_zip, Length: 9215, dtype: int64

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
columns_to_drop = ['vin', 'franchise_dealer']
new_df = test.drop(columns=columns_to_drop, axis=0)
new_df.head()

In [ ]:
# Determine the number of unique values in each column.
new_df.info()

In [ ]:
new_df['daysonmarket'].value_counts()

In [ ]:
new_df.categorize()

In [ ]:
import dask.dataframe as dd
from dask.distributed import Client

# Create a Dask Client (optional but recommended for better performance)
client = Client()

# Use dask.dataframe.get_dummies() to one-hot encode categorical columns
application_dummies = dd.get_dummies(new_df)

# Compute the result and convert it to a pandas DataFrame
application_dummies = application_dummies.compute()

# Display the result
print(application_dummies)
